<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np

# Загрузка данных
trips = pd.read_csv('trips.csv')  # №_машины, Дата_Время, Широта, Долгота
bus_lanes = gpd.read_file('bus_lanes.geojson')  # Выделенные полосы

# Преобразование поездок в геометрию
trips['geometry'] = trips.apply(lambda row: Point(row['Долгота'], row['Широта']), axis=1)
trips_gdf = gpd.GeoDataFrame(trips, geometry='geometry')

# Функция для поиска пересечений траектории с полосами
def find_overlap(trajectory, bus_lane):
    buffer = bus_lane.buffer(0.0005)  # ~50 метров в десятичных градусах
    overlap = trajectory.within(buffer)
    return overlap

# Группировка по машинам и построение траекторий
for car_id, group in trips_gdf.groupby('№_машины'):
    group = group.sort_values('Дата_Время')
    trajectory = LineString(group['geometry'].tolist())

    # Проверка пересечения с каждой выделенной полосой
    for idx, lane in bus_lanes.iterrows():
        if find_overlap(trajectory, lane['geometry']):
            # Вычисление времени и длины сегмента
            segment = group[group.within(lane['geometry'].buffer(0.0005))]
            time_current = (pd.to_datetime(segment['Дата_Время'].iloc[-1]) -
                           pd.to_datetime(segment['Дата_Время'].iloc[0])).total_seconds() / 3600
            length = lane['length'] / 1000  # в км
            speed_bus_lane = 50  # км/ч (пример)
            time_bus_lane = length / speed_bus_lane
            time_gain = time_current - time_bus_lane
            print(f"Машина {car_id}, сегмент {idx}: выигрыш {time_gain:.2f} часов")